<a href="https://colab.research.google.com/github/maudlcrf/rcg/blob/main/eda_post_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### AI for Retail & Consumer Goods | Group Project | MBD April 2024

Group Members: Maud Lecerf | Cristina Mosquera | Christopher Stephan

# Imports & Color Palette

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from google.colab import drive

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# Custom Theme
iowa = {
    "layout": {
        "colorway": ["#4c3624", "#ff0651", "#ff7e06", "#74B36B", "#628395",
                     "#996C48", "#FF5C8D", "#FF9633", "#8CC084", "#84A0AE",
                     "#B78B66", "#FF85A9", "#FFB570", "#A7CEA1", "#A9BCC7"],
        "plot_bgcolor": "white",
        "paper_bgcolor": "white",
        "font": {"color": "black"},
        "xaxis": {"gridcolor": "lightgray"},
        "yaxis": {"gridcolor": "lightgray"},
    }
}

# Register themes
pio.templates["iowa"] = iowa

In [4]:
#data_post_covid = pd.read_csv("G:\\Drive partagés\\RCG\\Iowa_Liquor_Sales_Post_Covid.csv") #Maud

drive.mount('/content/drive') #Cris
data_post_covid = pd.read_csv("/content/drive/MyDrive/Iowa_Liquor_Sales_Post_Covid.csv") #Cris

Mounted at /content/drive


## Splitting the data into alcohol types

In [5]:
## Regrouping the original categories into alcohol types
# Define mapping of keywords to subcategories
category_mapping = {
    'RUM': 'RUM',
    'VODKA': 'VODKAS',
    'VODKAS': 'VODKAS',
    'SCHNAPPS': 'SCHNAPPS',
    'BRANDIES': 'BRANDIES',
    'WHISKIES': 'WHISKIES',
    'SCOTCH': 'WHISKIES',
    'GINS': 'GINS',
    'GIN': 'GINS',
    'MEZCAL': 'MEZCAL',
    'TEQUILA': 'TEQUILA',
    'BOURBON': 'BOURBON',
    'LIQUEURS': 'LIQUEURS',
    'LIQUEUR':'LIQUEURS',
    'AMARETTO': 'AMARETTO',
    'CREME': 'CREME'
}

# Create a new column and assign "OTHERS" by default
data_post_covid['alcohol_type'] = 'OTHERS'

# Loop through mapping and assign subcategories
for keyword, alcohol_type in category_mapping.items():
    data_post_covid.loc[data_post_covid['category_name'].str.contains(keyword, case=False, na=False, regex=True), 'alcohol_type'] = alcohol_type



In [ ]:
data_post_covid.nunique()

invoice_item_number      10414198
date                         1213
store_number                 2445
store_name                   2515
address                      2531
city                          477
zip_code                      514
store_location              10803
county_number                  99
county                         99
category                       59
category_name                  48
vendor_number                 345
vendor_name                   357
item_number                  8278
item_description             7554
pack                           22
bottle_volume_(ml)             26
state_bottle_cost            2231
state_bottle_retail          2235
bottles_sold                  610
sale_(dollars)              20487
volume_sold_(liters)         1227
volume_sold_(gallons)        1214
alcohol_type                   11
dtype: int64

In [ ]:
data_post_covid['alcohol_type'].value_counts()

alcohol_type
VODKAS      2481955
WHISKIES    2093746
LIQUEURS    1377236
RUM          903040
OTHERS       849708
BOURBON      842461
TEQUILA      686286
SCHNAPPS     486618
BRANDIES     421626
GINS         260273
MEZCAL        11249
Name: count, dtype: int64

In [6]:
data_post_covid['category_name'].value_counts()

,count
category_name,
AMERICAN VODKAS,1595428
CANADIAN WHISKIES,976635
STRAIGHT BOURBON WHISKIES,787101
WHISKEY LIQUEUR,630916
AMERICAN FLAVORED VODKA,490908
100% AGAVE TEQUILA,461341
SPICED RUM,415297
BLENDED WHISKIES,389725
AMERICAN SCHNAPPS,341068


In [7]:
#separating the columns into numerical and categorical as it my be helpful later on
numerical_cols = ['pack', 'bottle_volume_(ml)', 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 'sale_(dollars)',
                  'volume_sold_(liters)', 'volume_sold_(gallons)' ]
categorical_cols = ['invoice_item_number', 'date', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'store_location', 'county_number', 'county',
       'category', 'category_name', 'vendor_number', 'vendor_name',
       'item_number', 'item_description' ]

In [8]:
data_post_covid['date'] = pd.to_datetime(data_post_covid['date'])

# Create new columns
data_post_covid['year'] = data_post_covid['date'].dt.year
data_post_covid['month'] = data_post_covid['date'].dt.month
data_post_covid['year_month'] = data_post_covid['date'].dt.to_period('M')
data_post_covid['weekday'] = data_post_covid['date'].dt.weekday

# Grouping data by alcohol
data_yearly_alcohol = data_post_covid.groupby(['year', 'alcohol_type'])[numerical_cols].sum().reset_index()
data_monthly_alcohol = data_post_covid.groupby(['year', 'month', 'alcohol_type'])[numerical_cols].sum().reset_index()
data_year_month_alcohol = data_post_covid.groupby(['year_month', 'alcohol_type'])[numerical_cols].sum().reset_index()
data_weekday_alcohol = data_post_covid.groupby(['weekday', 'alcohol_type'])[numerical_cols].sum().reset_index()

In [9]:
# Grouping data by category
data_yearly_category = data_post_covid.groupby(['year', 'category_name'])[numerical_cols].sum().reset_index()
data_monthly_category = data_post_covid.groupby(['year', 'month', 'category_name'])[numerical_cols].sum().reset_index()
data_year_month_category = data_post_covid.groupby(['year_month', 'category_name'])[numerical_cols].sum().reset_index()
data_weekday_category = data_post_covid.groupby(['weekday', 'category_name'])[numerical_cols].sum().reset_index()

In [ ]:
px.bar(data_yearly, x='year', y='volume_sold_(liters)', color= 'alcohol_type', template = iowa)

In [ ]:
px.line(data_yearly, x='year', y='volume_sold_(liters)', color= 'alcohol_type', template = iowa)

In [ ]:
px.bar(data_yearly, x='year', y='bottles_sold', color= 'alcohol_type', template = iowa)

In [ ]:
px.line(data_yearly, x='year', y='bottles_sold', color= 'alcohol_type', template = iowa)

In [ ]:
px.bar(data_yearly, x='year', y='sale_(dollars)', color= 'alcohol_type', template = iowa)

In [ ]:
px.line(data_yearly, x='year', y='sale_(dollars)', color= 'alcohol_type', template = iowa)

In [11]:
#Creating new features to understand the performance of each cateogry

# Price related features
data_post_covid['price_per_liter'] = data_post_covid['state_bottle_retail'] / data_post_covid['bottle_volume_(ml)'] * 1000
data_post_covid['profit_per_bottle'] = data_post_covid['state_bottle_retail'] - data_post_covid['state_bottle_cost']
data_post_covid['profit_margin'] = (data_post_covid['profit_per_bottle'] / data_post_covid['state_bottle_retail']) * 100


# Sales performance features
data_post_covid['sales_per_store'] = data_post_covid.groupby(['store_number', 'year_month'])['sale_(dollars)'].transform('sum')


# Time based features
data_post_covid['is_weekend'] = data_post_covid['weekday'].isin([5,6]).astype(int)
data_post_covid['quarter'] = data_post_covid['date'].dt.quarter

In [ ]:
# Plotting sales for each category
fig_sales_category = px.bar(data_post_covid,
                           x='category_name',
                           y='sale_(dollars)',
                           color='category_name',
                           title='Total Sales by Category',
                           template='iowa')
fig_sales_category.show()


In [ ]:
# Plotting sales for each alcohol type
fig_sales_alcohol_type = px.bar(data_post_covid,
                           x='alcohol_type',
                           y='sale_(dollars)',
                           color='alcohol_type',
                           title='Total Sales by Alcohol Type',
                           template='iowa')
fig_sales_alcohol_type.show()

In [ ]:
# Plotting sales trend over time for each category (using year_month)
fig_sales_trend = px.line(data_year_month,
                         x='year_month',
                         y='sale_(dollars)',
                         color='alcohol_type',
                         title='Sales Trend by Alcohol Type Over Time',
                         template='iowa')
fig_sales_trend.show()


In [ ]:
# Plotting sales over time for each alcohol_type
for alcohol_type in data_post_covid['alcohol_type'].unique():
  subset = data_post_covid[data_post_covid['alcohol_type']==alcohol_type]
  fig = px.line(subset, x='date', y='sale_(dollars)', title=f'Sales of {alcohol_type} over time')
  fig.show()


#Sales

In [13]:
# Evolutions of sales per category
fig = px.line(data_yearly_category,
              x='year',
              y='sale_(dollars)',
              color='category_name',
              title='Evolution of Sales by Category per Year',
              template='iowa')
fig.show()


Since we cannot see any peaks of find a specific pattern, we'll try to find it in the % growth of sales.

In [25]:
# Calculate year-over-year sales growth for each category
sales_growth = data_yearly_category.copy()
sales_growth['sales_growth'] = sales_growth.groupby('category_name')['sale_(dollars)'].pct_change() * 100

# Fill NaN values with 0
sales_growth['sales_growth'] = sales_growth['sales_growth'].fillna(0)

# Display the sales growth
print(sales_growth[['year', 'category_name', 'sale_(dollars)', 'sales_growth']])


     year                         category_name  sale_(dollars)  sales_growth
0    2021                    100% AGAVE TEQUILA     23915100.71      0.000000
1    2021                         AGED DARK RUM      1073387.53      0.000000
2    2021                     AMERICAN BRANDIES      5216033.32      0.000000
3    2021          AMERICAN CORDIALS & LIQUEURS      4293640.07      0.000000
4    2021  AMERICAN DISTILLED SPIRITS SPECIALTY      1497977.38      0.000000
..    ...                                   ...             ...           ...
178  2024        TEMPORARY & SPECIALTY PACKAGES     16366569.75     72.270271
179  2024                    TENNESSEE WHISKIES     16886917.43      0.535197
180  2024                            TRIPLE SEC       822715.98     -5.567713
181  2024                       WHISKEY LIQUEUR     26510560.02     -0.078423
182  2024                             WHITE RUM      5500138.30     -6.744507

[183 rows x 4 columns]


In [26]:
# Plotting the growth of each category
fig_sales_growth = px.line(sales_growth,
                           x='year',
                           y='sales_growth',
                           color='category_name',
                           title='Year-over-Year Sales Growth by Category',
                           template='iowa')
fig_sales_growth.show()


it's a lot of information so it's hard to analyze, let's filter by the % of fluctuation, to see the categories that have growth or decrease their sales


In [43]:
# prompt: from sales_growth show me the most fluctuating categories

# Calculate the absolute value of sales growth
sales_growth['abs_sales_growth'] = abs(sales_growth['sales_growth'])

# Find categories with the highest absolute sales growth
most_fluctuating = sales_growth.groupby('category_name')['abs_sales_growth'].mean().sort_values(ascending=False)

# Threshold 20% of fluctuation
threshold = 20
highly_fluctuating_categories = sales_growth[sales_growth['abs_sales_growth'] > threshold]
print("\nCategories with absolute sales growth greater than", threshold, "%:")
highly_fluctuating_categories



Categories with absolute sales growth greater than 20 %:


,year,category_name,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),volume_sold_(gallons),sales_growth,abs_sales_growth
59,2022,BOTTLED IN BOND BOURBON,81024,7736000,194588.44,291886.94,33086,951488.73,28757.25,7541.87,36.963724,36.963724
65,2022,DISTILLED SPIRITS SPECIALTY,168,21000,674.90,1012.49,100,3564.98,75.00,19.61,-87.299679,87.299679
66,2022,FLAVORED GIN,23850,2386400,53139.98,79712.90,28305,683890.86,20855.65,5485.84,76.234850,76.234850
74,2022,IMPORTED GINS,60,3750,73.75,110.65,72,1593.36,54.00,14.23,-50.000000,50.000000
77,2022,IMPORTED WHISKIES,12,1500,127.84,191.76,12,1150.56,9.00,2.36,-79.151990,79.151990
82,2022,NEUTRAL GRAIN SPIRITS FLAVORED,210900,16469150,364193.80,546340.05,168258,2824263.02,91560.30,23967.06,79.844874,79.844874
84,2022,SINGLE BARREL BOURBON WHISKIES,33438,3847125,122110.79,183169.19,37400,1337022.01,28086.00,7378.87,25.374804,25.374804
90,2022,TEMPORARY & SPECIALTY PACKAGES,634452,53786500,1970429.81,2955790.43,575317,15366045.72,455046.19,119678.28,51.087251,51.087251
99,2023,AMERICAN DISTILLED SPIRITS SPECIALTY,97636,7677750,155180.38,232783.84,95909,2118431.67,68686.10,18072.59,39.608579,39.608579
106,2023,BOTTLED IN BOND BOURBON,87675,8610300,223678.02,335519.16,41890,1266568.55,36132.30,9481.85,33.114404,33.114404


In [45]:
# Filter data for years starting from 2021
sales_growth_2021 = sales_growth[sales_growth['year'] >= 2021]

# Plotting the growth of highly fluctuating categories from 2021 onwards
fig_highly_fluctuating = px.line(sales_growth_2021[sales_growth_2021['category_name'].isin(highly_fluctuating_categories['category_name'])],
                                 x='year',
                                 y='sales_growth',
                                 color='category_name',
                                 title='Year-over-Year Sales Growth of Highly Fluctuating Categories (2021-2024)',
                                 template='iowa')
fig_highly_fluctuating.show()


The following categories don't have a continuis line, they stop at year 2022. Let's understand waht hapenned
- imported gins
- imported whiskies
- destilled spirits specialty

In [29]:
# Filter data for specific categories
imported_gins = data_yearly_category[data_yearly_category['category_name'] == 'IMPORTED GINS']
imported_whiskies = data_yearly_category[data_yearly_category['category_name'] == 'IMPORTED WHISKIES']
distilled_spirits = data_yearly_category[data_yearly_category['category_name'] == 'DISTILLED SPIRITS SPECIALTY']

# Check if data exists for each category
if not imported_gins.empty:
    print("Data available for Imported Gins:")
    print(imported_gins[['year', 'sale_(dollars)']])
else:
    print("No data found for Imported Gins.")

if not imported_whiskies.empty:
    print("\nData available for Imported Whiskies:")
    print(imported_whiskies[['year', 'sale_(dollars)']])
else:
    print("No data found for Imported Whiskies.")

if not distilled_spirits.empty:
    print("\nData available for Distilled Spirits Specialty:")
    print(distilled_spirits[['year', 'category_name', 'sale_(dollars)']])
else:
    print("No data found for Distilled Spirits Specialty.")


Data available for Imported Gins:
    year  sale_(dollars)
27  2021         3186.72
74  2022         1593.36

Data available for Imported Whiskies:
    year  sale_(dollars)
30  2021         5518.80
77  2022         1150.56

Data available for Distilled Spirits Specialty:
    year                category_name  sale_(dollars)
18  2021  DISTILLED SPIRITS SPECIALTY        28070.00
65  2022  DISTILLED SPIRITS SPECIALTY         3564.98


Since we dont have data of these 3 categories for the last 2 years, we are not going to take them into consideration in the post covid analysis.

Back to the fluctuations in sales growth. Is this fluctuation a generalization of their alcohol type or specific to the category?